In [7]:
#all da imports
import pandas as pd

In [ ]:
# each row is a festival screening of a film (so films shown multiple times have multiple rows)
fest_long = pd.read_csv("data/1_film-dataset_festival-program_long.csv")
# each row is a film, where the data for the first festival it was shown at listed
fest_wide = pd.read_csv("data/1_film-dataset_festival-program_wide.csv")
#  each row is an award of a given film, including nominations and wins
imdb_awards = pd.read_csv("data/3_imdb-dataset_awards_long.csv")

imdb_awards

,imdb.id,award.institution,imdb.event.id,award.imdb.year,awards.nom.win,award,award.category,award.person,imdb.name.id
0,tt0019760,"Cinema Eye Honors Awards, US",ev0001823,2014,Winner,Cinema Eye Honors Award,The Influentials,Dziga Vertov,nm0895048
1,tt0019760,Village Voice Film Poll,ev0002738,1999,Nominee,VVFP Award,Best Film of the Century,NaN,NaN
2,tt0024844,Village Voice Film Poll,ev0002738,1999,Nominee,VVFP Award,Best Film of the Century,NaN,NaN
3,tt0025456,"National Board of Review, USA",ev0000464,1934,Winner,NBR Award,Best Foreign Film,NaN,NaN
4,tt0025456,"National Board of Review, USA",ev0000464,1934,Winner,NBR Award,Top Foreign Films,NaN,NaN
...,...,...,...,...,...,...,...,...,...
106006,tt3093520,Palm Springs International Film Festival,ev0000521,2014,Nominee,Audience Award,Best Documentary Feature,Guido Santi,nm1127368
106007,tt2893194,Chicago International Film Festival,ev0000165,2013,Winner,Special Mention,Best Documentary,Huijing Xu,nm5663236
106008,tt2893194,Jeonju Film Festival,ev0001528,2013,Nominee,Grand Prize,International Competition,Huijing Xu,nm5663236
106009,tt2893194,Sheffield International Documentary Festival,ev0001360,2013,Winner,Special Mention,NaN,Huijing Xu,nm5663236
